In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive"


'10th &  12th Marksheet'	'Anindya_RESUME Final (6).pdf'
'Anindya_CV final.pdf'		'Anindya_RESUME Final (7).pdf'
 Anindya_CV.pdf			'Anindya_RESUME Final.pdf'
'ANINDYA CV.pdf'		'Colab Notebooks'
 Anindya_Mishra.pdf		 Kaggle_Anomaly_Detection
'Anindya_RESUME Final (1).pdf'	 lendingclub_clean_dev.csv
'Anindya_RESUME Final (2).pdf'	 lendingclub_clean_full.csv
'Anindya_RESUME Final (3).pdf'	 MeetMux
'Anindya_RESUME Final (4).pdf'	'MyWays.ai RESUME LINK'
'Anindya_RESUME Final (5).pdf'	'NEENOPAL RESUME LINK'


In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split


In [7]:
df = pd.read_csv('/content/drive/MyDrive/lendingclub_clean_dev.csv')
print(df.shape)
df.head()


(20000, 125)


,loan_amnt,int_rate,annual_inc,loan_status,dti,fico_range_low,fico_range_high,target,term_ 60 months,purpose_credit_card,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_Unknown,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,-0.733929,-0.684682,-0.477200,Fully Paid,1.312448,-0.785008,-0.784989,0,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0.316555,1.080772,-0.072932,Charged Off,0.409401,0.646726,0.646699,1,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-1.518906,-0.013714,-0.278083,Fully Paid,-0.870915,-0.148682,-0.148684,0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,-0.272178,1.071254,-0.097068,Fully Paid,1.709088,-0.785008,-0.784989,0,True,False,...,False,True,False,False,False,False,False,False,False,False
4,-1.311118,0.386011,-0.519437,Fully Paid,1.807787,0.010400,0.010393,0,False,False,...,False,True,False,False,False,False,False,False,False,False


In [10]:
print(df['target'].unique())
print(df.dtypes)


[0. 1.]
loan_amnt        float64
int_rate         float64
annual_inc       float64
loan_status       object
dti              float64
                  ...   
addr_state_VT       bool
addr_state_WA       bool
addr_state_WI       bool
addr_state_WV       bool
addr_state_WY       bool
Length: 125, dtype: object


In [11]:
# Drop leftover categorical original column
df = df.drop(columns=['loan_status'], errors='ignore')

# Convert boolean columns to float
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(np.float32)

# Convert remaining to float
df = df.astype(np.float32)


In [12]:
print(df.dtypes)
print(df.head())


loan_amnt         float32
int_rate          float32
annual_inc        float32
dti               float32
fico_range_low    float32
                   ...   
addr_state_VT     float32
addr_state_WA     float32
addr_state_WI     float32
addr_state_WV     float32
addr_state_WY     float32
Length: 124, dtype: object
   loan_amnt  int_rate  annual_inc       dti  fico_range_low  fico_range_high  \
0  -0.733929 -0.684682   -0.477200  1.312448       -0.785008        -0.784989   
1   0.316555  1.080772   -0.072932  0.409401        0.646726         0.646699   
2  -1.518906 -0.013714   -0.278083 -0.870915       -0.148682        -0.148684   
3  -0.272178  1.071254   -0.097068  1.709088       -0.785008        -0.784989   
4  -1.311118  0.386011   -0.519437  1.807787        0.010400         0.010393   

   target  term_ 60 months  purpose_credit_card  purpose_debt_consolidation  \
0     0.0              0.0                  0.0                         1.0   
1     1.0              0.0                

In [18]:
df['target'].value_counts(normalize=True)


,proportion
target,
0.0,0.78965
1.0,0.21035


In [ ]:
#TABNET CODE

In [40]:
!pip install pytorch-tabnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.8 MB/s eta 0:00:00


In [47]:
import numpy as np
import pandas as pd
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight


In [48]:
X = df.drop("target", axis=1).astype(np.float32).values
y = df["target"].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [49]:
# Compute class weights
class_w = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

# Map to sample weights
sample_weights = np.where(y_train == 0, class_w[0], class_w[1]).astype(np.float32)

print("Sample Weights:", sample_weights[:10])


Sample Weights: [0.633212  0.633212  0.633212  2.3767083 0.633212  0.633212  0.633212
 0.633212  0.633212  0.633212 ]


In [50]:
tabnet = TabNetClassifier(
    n_d=24, n_a=24,
    n_steps=5,
    gamma=1.3,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    mask_type="entmax",
)


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [51]:
tabnet.fit(
    X_train=X_train,
    y_train=y_train,
    max_epochs=40,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    weights=sample_weights  # <-- correct parameter
)


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.14363 |  0:00:02s
epoch 1  | loss: 1.08604 |  0:00:04s
epoch 2  | loss: 1.02792 |  0:00:06s
epoch 3  | loss: 0.98283 |  0:00:08s
epoch 4  | loss: 0.96279 |  0:00:10s
epoch 5  | loss: 0.90089 |  0:00:12s
epoch 6  | loss: 0.86013 |  0:00:14s
epoch 7  | loss: 0.83269 |  0:00:16s
epoch 8  | loss: 0.82449 |  0:00:18s
epoch 9  | loss: 0.79734 |  0:00:20s
epoch 10 | loss: 0.78927 |  0:00:22s
epoch 11 | loss: 0.77976 |  0:00:25s
epoch 12 | loss: 0.76468 |  0:00:26s
epoch 13 | loss: 0.74819 |  0:00:28s
epoch 14 | loss: 0.74529 |  0:00:31s
epoch 15 | loss: 0.71764 |  0:00:33s
epoch 16 | loss: 0.72273 |  0:00:35s
epoch 17 | loss: 0.71359 |  0:00:36s
epoch 18 | loss: 0.70651 |  0:00:38s
epoch 19 | loss: 0.69554 |  0:00:40s
epoch 20 | loss: 0.69828 |  0:00:42s
epoch 21 | loss: 0.68546 |  0:00:45s
epoch 22 | loss: 0.6888  |  0:00:47s
epoch 23 | loss: 0.67544 |  0:00:48s
epoch 24 | loss: 0.67112 |  0:00:50s
epoch 25 | loss: 0.66887 |  0:00:52s
epoch 26 | loss: 0.66997 |  0:00:54s
e

In [52]:
probs = tabnet.predict_proba(X_test)[:, 1]
preds = (probs >= 0.5).astype(int)

print("Accuracy :", accuracy_score(y_test, preds))
print("F1 Score :", f1_score(y_test, preds))
print("ROC-AUC  :", roc_auc_score(y_test, probs))


Accuracy : 0.653
F1 Score : 0.4509493670886076
ROC-AUC  : 0.7163960509184449
